# Camera Status and Availability Testing

This notebook tests various methods to check camera status and availability before attempting to capture images.

## Key Functions to Test:
- `CameraIsOpened()` - Check if camera is already opened by another process
- `CameraInit()` - Try to initialize and catch specific error codes
- `CameraGetImageBuffer()` - Test actual image capture with timeout
- Error code analysis for different failure scenarios

In [46]:
# Import required libraries
import os
import sys
import time
import numpy as np
import cv2
import platform
from datetime import datetime

# Add the python demo directory to path to import mvsdk
sys.path.append('./python demo')
import mvsdk

print("Libraries imported successfully!")
print(f"Platform: {platform.system()}")

Libraries imported successfully!
Platform: Windows


In [47]:
# Let's examine the relevant error codes from the SDK
print("Relevant Camera Status Error Codes:")
print("=" * 40)
print(f"CAMERA_STATUS_SUCCESS: {mvsdk.CAMERA_STATUS_SUCCESS}")
print(f"CAMERA_STATUS_DEVICE_IS_OPENED: {mvsdk.CAMERA_STATUS_DEVICE_IS_OPENED}")
print(f"CAMERA_STATUS_DEVICE_IS_CLOSED: {mvsdk.CAMERA_STATUS_DEVICE_IS_CLOSED}")
print(f"CAMERA_STATUS_ACCESS_DENY: {mvsdk.CAMERA_STATUS_ACCESS_DENY}")
print(f"CAMERA_STATUS_DEVICE_LOST: {mvsdk.CAMERA_STATUS_DEVICE_LOST}")
print(f"CAMERA_STATUS_TIME_OUT: {mvsdk.CAMERA_STATUS_TIME_OUT}")
print(f"CAMERA_STATUS_BUSY: {mvsdk.CAMERA_STATUS_BUSY}")
print(f"CAMERA_STATUS_NO_DEVICE_FOUND: {mvsdk.CAMERA_STATUS_NO_DEVICE_FOUND}")

Relevant Camera Status Error Codes:
CAMERA_STATUS_SUCCESS: 0
CAMERA_STATUS_DEVICE_IS_OPENED: -18
CAMERA_STATUS_DEVICE_IS_CLOSED: -19
CAMERA_STATUS_ACCESS_DENY: -45
CAMERA_STATUS_DEVICE_LOST: -38
CAMERA_STATUS_TIME_OUT: -12
CAMERA_STATUS_BUSY: -28
CAMERA_STATUS_NO_DEVICE_FOUND: -16


In [48]:
def check_camera_availability():
    """
    Comprehensive camera availability check
    """
    print("Camera Availability Check")
    print("=" * 30)
    
    # Step 1: Initialize SDK
    try:
        mvsdk.CameraSdkInit(1)
        print("✓ SDK initialized successfully")
    except Exception as e:
        print(f"✗ SDK initialization failed: {e}")
        return None, "SDK_INIT_FAILED"
    
    # Step 2: Enumerate cameras
    try:
        DevList = mvsdk.CameraEnumerateDevice()
        nDev = len(DevList)
        print(f"✓ Found {nDev} camera(s)")
        
        if nDev < 1:
            print("✗ No cameras detected")
            return None, "NO_CAMERAS"
            
        for i, DevInfo in enumerate(DevList):
            print(f"  {i}: {DevInfo.GetFriendlyName()} ({DevInfo.GetPortType()})")
            
    except Exception as e:
        print(f"✗ Camera enumeration failed: {e}")
        return None, "ENUM_FAILED"
    
    # Step 3: Check all cameras
    camera_results = []
    
    for i, DevInfo in enumerate(DevList):
        print(f"\nTesting camera {i}: {DevInfo.GetFriendlyName()}")
        
        # Check if camera is already opened
        try:
            is_opened = mvsdk.CameraIsOpened(DevInfo)
            if is_opened:
                print("✗ Camera is already opened by another process")
                camera_results.append((DevInfo, "ALREADY_OPENED"))
                continue
            else:
                print("✓ Camera is available (not opened by another process)")
        except Exception as e:
            print(f"⚠ Could not check if camera is opened: {e}")
        
        # Try to initialize camera
        try:
            hCamera = mvsdk.CameraInit(DevInfo, -1, -1)
            print("✓ Camera initialized successfully")
            camera_results.append((hCamera, "AVAILABLE"))
            
            # Close the camera after testing
            try:
                mvsdk.CameraUnInit(hCamera)
                print("✓ Camera closed after testing")
            except Exception as e:
                print(f"⚠ Warning: Could not close camera: {e}")
            
        except mvsdk.CameraException as e:
            print(f"✗ Camera initialization failed: {e.error_code} - {e.message}")
            
            # Analyze specific error codes
            if e.error_code == mvsdk.CAMERA_STATUS_DEVICE_IS_OPENED:
                camera_results.append((DevInfo, "DEVICE_OPENED"))
            elif e.error_code == mvsdk.CAMERA_STATUS_ACCESS_DENY:
                camera_results.append((DevInfo, "ACCESS_DENIED"))
            elif e.error_code == mvsdk.CAMERA_STATUS_DEVICE_LOST:
                camera_results.append((DevInfo, "DEVICE_LOST"))
            else:
                camera_results.append((DevInfo, f"INIT_ERROR_{e.error_code}"))
        
        except Exception as e:
            print(f"✗ Unexpected error during initialization: {e}")
            camera_results.append((DevInfo, "UNEXPECTED_ERROR"))
    
    return camera_results

# Test the function
camera_results = check_camera_availability()
print(f"\nResults for {len(camera_results)} cameras:")
for i, (camera_info, status) in enumerate(camera_results):
    if hasattr(camera_info, 'GetFriendlyName'):
        name = camera_info.GetFriendlyName()
    else:
        name = f"Camera {i}"
    print(f"  {name}: {status}")

Camera Availability Check
✓ SDK initialized successfully
✓ Found 2 camera(s)
  0: Blower-Yield-Cam (NET-1000M-192.168.1.165)
  1: Cracker-Cam (NET-1000M-192.168.1.167)

Testing camera 0: Blower-Yield-Cam
✓ Camera is available (not opened by another process)
✓ Camera initialized successfully
✓ Camera closed after testing

Testing camera 1: Cracker-Cam
✓ Camera is available (not opened by another process)
✓ Camera initialized successfully
✓ Camera closed after testing

Results for 2 cameras:
  Camera 0: AVAILABLE
  Camera 1: AVAILABLE


In [49]:
def test_camera_capture_readiness(hCamera):
    """
    Test if camera is ready for image capture
    """
    if not isinstance(hCamera, int):
        print("Camera not properly initialized, skipping capture test")
        return False
        
    print("\nTesting Camera Capture Readiness")
    print("=" * 35)
    
    try:
        # Get camera capabilities
        cap = mvsdk.CameraGetCapability(hCamera)
        print("✓ Camera capabilities retrieved")
        
        # Check camera type
        monoCamera = (cap.sIspCapacity.bMonoSensor != 0)
        print(f"✓ Camera type: {'Monochrome' if monoCamera else 'Color'}")
        
        # Set basic configuration
        if monoCamera:
            mvsdk.CameraSetIspOutFormat(hCamera, mvsdk.CAMERA_MEDIA_TYPE_MONO8)
        else:
            mvsdk.CameraSetIspOutFormat(hCamera, mvsdk.CAMERA_MEDIA_TYPE_BGR8)
        
        mvsdk.CameraSetTriggerMode(hCamera, 0)  # Continuous mode
        mvsdk.CameraSetAeState(hCamera, 0)      # Manual exposure
        mvsdk.CameraSetExposureTime(hCamera, 5000)  # 5ms exposure
        print("✓ Basic camera configuration set")
        
        # Start camera
        mvsdk.CameraPlay(hCamera)
        print("✓ Camera started")
        
        # Allocate buffer
        FrameBufferSize = cap.sResolutionRange.iWidthMax * cap.sResolutionRange.iHeightMax * (1 if monoCamera else 3)
        pFrameBuffer = mvsdk.CameraAlignMalloc(FrameBufferSize, 16)
        print("✓ Frame buffer allocated")
        
        # Test image capture with short timeout
        print("\nTesting image capture...")
        try:
            pRawData, FrameHead = mvsdk.CameraGetImageBuffer(hCamera, 1000)  # 1 second timeout
            print(f"✓ Image captured successfully: {FrameHead.iWidth}x{FrameHead.iHeight}")
            
            # Process and release
            mvsdk.CameraImageProcess(hCamera, pRawData, pFrameBuffer, FrameHead)
            mvsdk.CameraReleaseImageBuffer(hCamera, pRawData)
            print("✓ Image processed and buffer released")
            
            capture_success = True
            
        except mvsdk.CameraException as e:
            print(f"✗ Image capture failed: {e.error_code} - {e.message}")
            
            if e.error_code == mvsdk.CAMERA_STATUS_TIME_OUT:
                print("  → Camera timeout - may be busy or not streaming")
            elif e.error_code == mvsdk.CAMERA_STATUS_DEVICE_LOST:
                print("  → Device lost - camera disconnected")
            elif e.error_code == mvsdk.CAMERA_STATUS_BUSY:
                print("  → Camera busy - may be used by another process")
                
            capture_success = False
        
        # Cleanup
        mvsdk.CameraAlignFree(pFrameBuffer)
        print("✓ Cleanup completed")
        
        return capture_success
        
    except Exception as e:
        print(f"✗ Capture readiness test failed: {e}")
        return False

# Test capture readiness for available cameras
available_cameras = [(cam, stat) for cam, stat in camera_results if stat == "AVAILABLE"]

if available_cameras:
    print(f"\nTesting capture readiness for {len(available_cameras)} available camera(s):")
    for i, (camera_handle, status) in enumerate(available_cameras):
        if hasattr(camera_handle, 'GetFriendlyName'):
            # This shouldn't happen for AVAILABLE cameras, but just in case
            print(f"\nCamera {i}: Invalid handle")
            continue
        
        print(f"\nTesting camera {i} capture readiness...")
        # Re-initialize the camera for testing since we closed it earlier
        try:
            # Find the camera info from the original results
            DevList = mvsdk.CameraEnumerateDevice()
            if i < len(DevList):
                DevInfo = DevList[i]
                hCamera = mvsdk.CameraInit(DevInfo, -1, -1)
                capture_ready = test_camera_capture_readiness(hCamera)
                print(f"Capture Ready for {DevInfo.GetFriendlyName()}: {capture_ready}")
                mvsdk.CameraUnInit(hCamera)
            else:
                print(f"Could not re-initialize camera {i}")
        except Exception as e:
            print(f"Error testing camera {i}: {e}")
else:
    print("\nNo cameras are available for capture testing")
    print("Camera statuses:")
    for i, (cam_info, status) in enumerate(camera_results):
        if hasattr(cam_info, 'GetFriendlyName'):
            name = cam_info.GetFriendlyName()
        else:
            name = f"Camera {i}"
        print(f"  {name}: {status}")


Testing capture readiness for 2 available camera(s):

Testing camera 0 capture readiness...

Testing Camera Capture Readiness
✓ Camera capabilities retrieved
✓ Camera type: Color
✓ Basic camera configuration set
✓ Camera started
✓ Frame buffer allocated

Testing image capture...
✓ Image captured successfully: 1280x1024
✓ Image processed and buffer released
✓ Cleanup completed
Capture Ready for Blower-Yield-Cam: True

Testing camera 1 capture readiness...

Testing Camera Capture Readiness
✓ Camera capabilities retrieved
✓ Camera type: Color
✓ Basic camera configuration set
✓ Camera started
✓ Frame buffer allocated

Testing image capture...
✓ Image captured successfully: 1280x1024
✓ Image processed and buffer released
✓ Cleanup completed
Capture Ready for Cracker-Cam: True


In [50]:
def comprehensive_camera_check():
    """
    Complete camera availability and readiness check
    Returns: (available, ready, handle_or_info, status_message)
    """
    # Check availability
    handle_or_info, status = check_camera_availability()
    
    available = status == "AVAILABLE"
    ready = False
    
    if available:
        # Test capture readiness
        ready = test_camera_capture_readiness(handle_or_info)
        
        # Close camera after testing
        try:
            mvsdk.CameraUnInit(handle_or_info)
            print("✓ Camera closed after testing")
        except:
            pass
    
    return available, ready, handle_or_info, status

# Run comprehensive check
print("\n" + "=" * 50)
print("COMPREHENSIVE CAMERA CHECK")
print("=" * 50)

available, ready, info, status_msg = comprehensive_camera_check()

print("\n" + "=" * 50)
print("FINAL RESULTS:")
print(f"Camera Available: {available}")
print(f"Capture Ready: {ready}")
print(f"Status: {status_msg}")
print("=" * 50)


COMPREHENSIVE CAMERA CHECK
Camera Availability Check
✓ SDK initialized successfully
✓ Found 2 camera(s)
  0: Blower-Yield-Cam (NET-1000M-192.168.1.165)
  1: Cracker-Cam (NET-1000M-192.168.1.167)

Testing camera 0: Blower-Yield-Cam
✓ Camera is available (not opened by another process)
✓ Camera initialized successfully
✓ Camera closed after testing

Testing camera 1: Cracker-Cam
✓ Camera is available (not opened by another process)
✓ Camera initialized successfully
✓ Camera closed after testing

FINAL RESULTS:
Camera Available: False
Capture Ready: False
Status: (34, 'AVAILABLE')


In [51]:
def is_camera_ready_for_capture():
    """
    Simple function to check if camera is ready for capture.
    Returns: (ready: bool, message: str, camera_info: object or None)
    
    This is the function you can use in your main capture script.
    """
    try:
        # Initialize SDK
        mvsdk.CameraSdkInit(1)
        
        # Enumerate cameras
        DevList = mvsdk.CameraEnumerateDevice()
        if len(DevList) < 1:
            return False, "No cameras found", None
        
        DevInfo = DevList[0]
        
        # Check if already opened
        try:
            if mvsdk.CameraIsOpened(DevInfo):
                return False, f"Camera '{DevInfo.GetFriendlyName()}' is already opened by another process", DevInfo
        except:
            pass  # Some cameras might not support this check
        
        # Try to initialize
        try:
            hCamera = mvsdk.CameraInit(DevInfo, -1, -1)
            
            # Quick capture test
            try:
                # Basic setup
                mvsdk.CameraSetTriggerMode(hCamera, 0)
                mvsdk.CameraPlay(hCamera)
                
                # Try to get one frame with short timeout
                pRawData, FrameHead = mvsdk.CameraGetImageBuffer(hCamera, 500)  # 0.5 second timeout
                mvsdk.CameraReleaseImageBuffer(hCamera, pRawData)
                
                # Success - close and return
                mvsdk.CameraUnInit(hCamera)
                return True, f"Camera '{DevInfo.GetFriendlyName()}' is ready for capture", DevInfo
                
            except mvsdk.CameraException as e:
                mvsdk.CameraUnInit(hCamera)
                if e.error_code == mvsdk.CAMERA_STATUS_TIME_OUT:
                    return False, "Camera timeout - may be busy or not streaming properly", DevInfo
                else:
                    return False, f"Camera capture test failed: {e.message}", DevInfo
                    
        except mvsdk.CameraException as e:
            if e.error_code == mvsdk.CAMERA_STATUS_DEVICE_IS_OPENED:
                return False, f"Camera '{DevInfo.GetFriendlyName()}' is already in use", DevInfo
            elif e.error_code == mvsdk.CAMERA_STATUS_ACCESS_DENY:
                return False, f"Access denied to camera '{DevInfo.GetFriendlyName()}'", DevInfo
            else:
                return False, f"Camera initialization failed: {e.message}", DevInfo
                
    except Exception as e:
        return False, f"Camera check failed: {str(e)}", None

# Test the simple function
print("\nTesting Simple Camera Ready Check:")
print("=" * 40)

ready, message, camera_info = is_camera_ready_for_capture()
print(f"Ready: {ready}")
print(f"Message: {message}")
if camera_info:
    print(f"Camera: {camera_info.GetFriendlyName()}")


Testing Simple Camera Ready Check:
Ready: True
Message: Camera 'Blower-Yield-Cam' is ready for capture
Camera: Blower-Yield-Cam


## Usage Example

Here's how you can integrate the camera status check into your capture script:

```python
# Before attempting to capture images
ready, message, camera_info = is_camera_ready_for_capture()

if not ready:
    print(f"Camera not ready: {message}")
    # Handle the error appropriately
    return False

print(f"Camera ready: {message}")
# Proceed with normal capture logic
```

## Key Findings

1. **`CameraIsOpened()`** - Checks if camera is opened by another process
2. **`CameraInit()` error codes** - Provide specific failure reasons
3. **Quick capture test** - Verifies camera is actually streaming
4. **Timeout handling** - Detects if camera is busy/unresponsive

The most reliable approach is to:
1. Check if camera exists
2. Check if it's already opened
3. Try to initialize it
4. Test actual image capture with short timeout
5. Clean up properly